# sEEG

![seeg-schema-front](img/seeg-schema-front.png)

![seeg](img/seeg_tf_pre_ictal2.PNG)

Require registration of T1 & an image with electrodes (T1, or CT ideally)

In [4]:
%%bash
sd=$SUBJECTS_DIR/junch
mri_convert $< $sd/seeg/ELEC.nii.gz --out_orientation RAS
flirt -in $sd/seeg/ELEC.nii.gz -ref $sd/mri/T1.RAS.nii.gz \
    -omat $sd/seeg/ELEC_to_T1.mat \
    -out $sd/seeg/ELEC_in_T1.nii.gz \
    $regopts

bash: line 2: /opt/freesurfer/subjects/junch/seeg/ELEC.nii.gz: No such file or directory
Image Exception : #22 :: ERROR: Could not open image /opt/freesurfer/subjects/junch/mri/T1.RAS
terminate called after throwing an instance of 'RBD_COMMON::BaseException'
bash: line 6:   757 Aborted                 flirt -in $sd/seeg/ELEC.nii.gz -ref $sd/mri/T1.RAS.nii.gz -omat $sd/seeg/ELEC_to_T1.mat -out $sd/seeg/ELEC_in_T1.nii.gz $regopts


Verifying the alignment: T1 in grey, CT in red/yellow

![ct-coreg](img/ct-coreg.png)

Mask the image with FreeSurfer's brain mask (`$SUBJECTS_DIR/$SUBJECT/mri/brain.mgz`)

![ct-coreg-2](img/ct-coreg-2.png)

The exact x-y-z coordinates of each contact can be determined, depending on the exact dataset. The end result is a sEEG sensor file,

In [5]:
!head junch_seeg.txt

B'0	-29.808314	-18.415402	-37.230811
B'1	-33.185887	-18.179505	-36.905408
B'2	-36.563460	-17.943609	-36.580004
B'3	-39.941032	-17.707712	-36.254601
B'4	-43.318605	-17.471816	-35.929198
B'5	-46.696178	-17.235919	-35.603795
B'6	-50.073750	-17.000023	-35.278392
B'7	-53.451323	-16.764127	-34.952988
B'8	-56.828896	-16.528230	-34.627585
B'9	-60.206468	-16.292334	-34.302182


which can be used to create an sEEG sensors object

In [24]:
from tvb.datatypes.sensors import SensorsInternal
import numpy as np

xyz = np.loadtxt('junch_seeg.txt', usecols=(1,2,3))

with open('junch_seeg.txt', 'r') as fd:
    labels = [line.split()[0] for line in fd.readlines()]

sens_seeg = SensorsInternal(
    locations=xyz,
    labels=labels,
    number_of_sensors=len(xyz)
)

This sensors object would be used for simulation as previously shown, and TVB will automatically compute a gain matrix using a simple analytic form.

OpenMEEG or FEM software can be used to construct more realistic sEEG forward models.